In [4]:
%%time
from pulp import *

Jobs = [1,2,3,4,5]

Ptime = {
    1:2,
    2:2,
    3:1,
    4:3,
    5:4
}

Dtime = {
    1:3,
    2:3,
    3:2,
    4:9,
    5:4
}
#T is the max time.
T = sum(Ptime.values())
prob = LpProblem("HW1", LpMinimize)
Time = list(range(T))

#Decision variables xjt and tardyjt
xjt = LpVariable.dicts("X", (Jobs, Time), 0, 1, LpInteger)
tardyjt = LpVariable.dicts("tardy", (Jobs, Time), 0, 1, LpInteger)

#If job j is completed before duetime, tardyjt is 1 otherwise 0.
for j in Jobs:
    for t in Time:
        if int(Ptime[j]) + t <= int(Dtime[j]):
            tardyjt[(j,t)] = 0
        else:
            tardyjt[(j,t)]= 1
            
#The objective function:            
prob += (
    lpSum([xjt[j][t]*tardyjt[(j,t)] for j in Jobs for t in Time])
)

#Every job is assigned once.
for j in Jobs:
    prob += (
        lpSum(xjt[j][t] for t in range(T- Ptime[j]+1)) == 1
    )
             
#Two jobs should not overlap.        
for j in Jobs:
        Jobsremoved = []
        for i in range(j-1):
            Jobsremoved.append(Jobs[i])
        for i in range(j,len(Jobs)):
            Jobsremoved.append(Jobs[i])
        for t in Time:
            for j2 in Jobsremoved:
                for t2 in Time:
                    if t+Ptime[j]-1>=t2 and t2>=t:
                        prob += (
                            lpSum([xjt[j][t] + xjt[j2][t2]]) <= 1
                        )
                        
status = prob.solve()
print(LpStatus[status])
print("The optimal solution is: ") 
print(value(prob.objective))
for j in Jobs:
    for t in Time:
        if value(xjt[j][t]) == 1:
            print("Job "+ str(j) + " starts at " + str(t) )

Optimal
The optimal solution is: 
2
Job 1 starts at 1
Job 2 starts at 3
Job 3 starts at 0
Job 4 starts at 5
Job 5 starts at 8
Wall time: 78.8 ms


In [5]:
%%time
from pulp import *

Jobs = [1,2,3,4,5]

Ptime = {
    1:2,
    2:2,
    3:1,
    4:3,
    5:4
}

Dtime = {
    1:3,
    2:3,
    3:2,
    4:9,
    5:4
}

#T is the max time.
T = sum(Ptime.values())
prob = LpProblem("HW1", LpMinimize)
Time = list(range(T))

#Decision variables xjt and tardyjt
xjt = LpVariable.dicts("X", (Jobs, Time), 0, 1, LpInteger)
tardyjt = LpVariable.dicts("tardy", (Jobs, Time), 0, 1, LpInteger)

#If job j is completed before duetime, tardyjt is 1 otherwise 0.
for j in Jobs:
    for t in Time:
        if int(Ptime[j]) + t <= int(Dtime[j]):
            tardyjt[(j,t)] = 0
        else:
            tardyjt[(j,t)]= 1
            
#The objective function:                        
prob += (
    lpSum([xjt[j][t]*tardyjt[(j,t)] for j in Jobs for t in Time])
)

#Every job is assigned once.
for j in Jobs:
    prob += (
        lpSum(xjt[j][t] for t in range(T-Ptime[j]+1)) == 1
    )
    
#Two jobs should not overlap.             
for t in Time:
    prob += (
        lpSum(xjt[j][i] for j in Jobs for i in range(max(0,t-Ptime[j]),t)) <= 1
    )

status = prob.solve()
print(LpStatus[status])
print("The optimal solution is: ") 
print(value(prob.objective))
for j in Jobs:
    for t in Time:
        if value(xjt[j][t]) == 1:
            print("Job "+ str(j) + " starts at " + str(t) )

Optimal
The optimal solution is: 
2
Job 1 starts at 10
Job 2 starts at 1
Job 3 starts at 0
Job 4 starts at 3
Job 5 starts at 6
Wall time: 30.9 ms


# The computation time is smaller than the other formula. The model with alternative formula is faster.